# 0. 引入依赖包
注意 cx_Oracle裤需要动态依赖

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import os
import re
from db_con.pcn_oracle_data import PCNDB
from tqdm import tqdm
os.getcwd()
import yaml


In [2]:
os.environ['LD_LIBRARY_PATH']

':/usr/local/cuda/lib64:/home/wzk/Oracle/instantclient_19_15'

# 1. 连接数据库

In [3]:
pcnTestDB = PCNDB('PCN_TEST','784427618', 'localhost:1521/LHR11G')
bus_inf = pcnTestDB.get_all_businesses()
cha_inf = pcnTestDB.get_all_channels()

# 2. 获取有效通路
数据处理

In [4]:
def get_stn_name(stn_id, pcnDB, stnDict):
    del_full_name = lambda x:x[0].split('/')[-1] if x[0] != None else None
    if stn_id not in stnDict.keys():
        stn_dict = pcnDB.get_station_inf(stn_id, tCols=['FULL_NAME', 'VOLTAGE_CLASS', 'DSP_LEVEL'])
        stn_name = del_full_name(stn_dict['FULL_NAME']) if stn_dict['FULL_NAME'] != None else None
        vol_class = del_full_name(stn_dict['VOLTAGE_CLASS']) if stn_dict['VOLTAGE_CLASS'] != None else None
        dsp_level = del_full_name(stn_dict['DSP_LEVEL']) if stn_dict['DSP_LEVEL'] != None else None
        stnDict[stn_id] = {
            'STN_ID': stn_id,
            'STN_NAME': stn_name,
            'VOLTAGE_CLASS': vol_class,
            'DSP_LEVEL': dsp_level
        }
    else:
        stn_name = stnDict[stn_id]['STN_NAME']
    
    return stn_name, stnDict


def ch_channel_name(channelInf, pcnDB):
    assert 'A_STATION' in channelInf.keys() and 'Z_STATION' in channelInf.keys()
    stnList_a = channelInf['A_STATION']
    stnList_z = channelInf['Z_STATION']

    stnList_a_name , stnList_z_name = [], []
    resDict = {}

    for idx, (s_a, s_z) in enumerate(zip(stnList_a, stnList_z)):
        s_a_name, resDict = get_stn_name(s_a, pcnDB, resDict)
        s_z_name, resDict = get_stn_name(s_z, pcnDB, resDict)

        stnList_a_name.append(s_a_name)
        stnList_z_name.append(s_z_name)

    channelInf['A_STATION_NAME'] = stnList_a_name
    channelInf['Z_STATION_NAME'] = stnList_z_name

    return channelInf, resDict


def del_channel_inf(channelInf, pcnDB):
    channelInf, stnDict = ch_channel_name(channelInf, pcnDB)
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']

    assert len(stnList_a) == len(stnList_z)
    idx = 0
    while idx < len(stnList_a):
        s_a = stnList_a[idx]
        s_z = stnList_z[idx]
        if s_a is None or s_z is None:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        elif s_a == s_z:
            for k in channelInf.keys():
                channelInf[k].pop(idx)
        else:
            idx += 1

    channelInfPd = pd.DataFrame(data=channelInf)
    return channelInfPd, stnDict


def con_business_inf(b_id, pcnDB, stnDict):
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z = get_stn_name(stnID_z, pcnDB, stnDict)
    busInfDict['A_STATION_NAME'] = stnName_a
    busInfDict['Z_STATION_NAME'] = stnName_z

    return busInfDict


def con_business_stn_inf(b_id, pcnDB, stnDict):
    resDitc={}
    busInfDict = pcnDB.get_business_inf(b_id)
    stnID_a, stnID_z = busInfDict['A_SITE_ID'][0], busInfDict['Z_SITE_ID'][0]

    stnName_a,_ = get_stn_name(stnID_a, pcnDB, stnDict)
    stnName_z,_ = get_stn_name(stnID_z, pcnDB, stnDict)
    resDitc['A_STATION_NAME'] = stnName_a
    resDitc['Z_STATION_NAME'] = stnName_z

    return resDitc

 
def del_business_inf(businessInf, pcnDB, stnDict=None):
    assert 'BUSINESS_ID' in businessInf.keys() and 'CHANNEL_ID' in businessInf
    busDict = {}
    for idx, busID in enumerate(businessInf['BUSINESS_ID']):
        if busID not in busDict.keys():
            busDict[busID] = [businessInf['CHANNEL_ID'][idx]]
        else:
            busDict[busID].append(businessInf['CHANNEL_ID'][idx])

    businessInfDict ={}
    for idx, b_id in enumerate(list(busDict.keys())):
        busInfDict = con_business_stn_inf(b_id, pcnDB, stnDict)
        businessInfDict[b_id] = busInfDict

    return busDict, businessInfDict


def generate_pcnMap(channelInf):
    pcnMap = nx.Graph()
    stnList_a = channelInf['A_STATION_NAME']
    stnList_z = channelInf['Z_STATION_NAME']
    channelIdList = channelInf['OBJ_ID']

    edgeDict = {}
    for idx, (c_id, s_a, s_z) in enumerate(zip(channelIdList, stnList_a, stnList_z)):
        edgeStr = '{}-{}'.format(s_a, s_z)
        edgeStrRe = '{}-{}'.format(s_z, s_a)
        if edgeStr in edgeDict.keys():
            edgeDict[edgeStr].append(c_id)
        elif edgeStrRe in edgeDict.keys():
            edgeDict[edgeStrRe].append(c_id)
        else:
            pcnMap.add_edge(s_a, s_z)
            edgeDict[edgeStr] = [c_id]

    for e in pcnMap.edges.keys():
        edgekey = '{}-{}'.format(e[0], e[1])
        edgekeyRe = '{}-{}'.format(e[1], e[0])
        if edgekey in edgeDict.keys():
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekey]
        else:
            pcnMap.edges[e]['channel_id'] = edgeDict[edgekeyRe]


    return pcnMap


def get_circuitous_bus(businessInfDict, pcnMap):
    busDelDict={}
    for idx,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            busDelDict[b_id] = businessInfDict[b_id]

    return busDelDict

def get_circuitous_channel(busDelDict, busDict):
    chaDelDict={}
    for b_id in busDelDict.keys():
        if b_id in busDict.keys():
            chaDelDict[b_id] = busDict[b_id]
    
    return chaDelDict

def get_circuitous_inf(busInfDict, businessInfDict, pcnMap):
    pcnBussInf={}
    for _,b_id in enumerate(businessInfDict.keys()):
        stnName_a = businessInfDict[b_id]['A_STATION_NAME']
        stnName_z = businessInfDict[b_id]['Z_STATION_NAME']

        if stnName_a not in pcnMap.nodes.keys() or stnName_z not in pcnMap.nodes.keys():
            continue
        elif stnName_a is None or stnName_z is None:
            continue
        elif stnName_a == stnName_z:
            continue
        else:
            pcnBussInf[b_id] = businessInfDict[b_id]
    
    for b_id in pcnBussInf.keys():
        if b_id in busInfDict.keys():
            pcnBussInf[b_id]['CHANNELS'] = busInfDict[b_id]
    
    return pcnBussInf



In [5]:
channelInfPd, stnDict = del_channel_inf(cha_inf, pcnTestDB)
busInfDict, businessInfDict = del_business_inf(bus_inf, pcnTestDB, stnDict)
pcnMap = generate_pcnMap(channelInfPd)
# busDelDict = get_circuitous_bus(businessInfDict, pcnMap)
# chaDelDict = get_circuitous_channel(busDelDict, busInfDict)
pcnBussInf =  get_circuitous_inf(busInfDict, businessInfDict, pcnMap)

## 2.1 获取所有可用通路
利用nx.all_simple_paths

In [6]:
def get_access_routes(pcnBussInf, pcnMap, cutoff=3):
    resDict={}
    routeDict = {}
    busIDList = list(pcnBussInf.keys())
    for idx in tqdm(range(len(pcnBussInf))):
        b_id = busIDList[idx]
        stn_a, stn_z = pcnBussInf[b_id]['A_STATION_NAME'], pcnBussInf[b_id]['Z_STATION_NAME']
        resKey = "{}-{}".format(stn_a, stn_z)
        resKey1 = "{}-{}".format(stn_z, stn_a)
        if resKey in resDict.keys():
            routeDict[b_id] = resDict[resKey]
        elif resKey1 in resDict.keys():
            routeDict[b_id] = resDict[resKey1]
        else:
            nx_allRoutes = nx.all_simple_paths(pcnMap,source=stn_a,target=stn_z, cutoff=cutoff)
            routes = list(nx_allRoutes)
            routeDict[b_id] = routes
            resDict[resKey] = routes

    return routeDict
    
routeDict = get_access_routes(pcnBussInf, pcnMap)

100%|██████████| 18083/18083 [00:20<00:00, 865.31it/s] 


# 3.路由优选

## 3.1 厂商筛选

In [7]:
def del_business_fac(chaList, pcnDB):
    assert isinstance(chaList, list)
    get_value = lambda x: x["NAME"][0] if 'NAME' in x.keys() else None

    facList = [get_value(pcnDB.get_fac_inf(c_id)) for c_id in chaList]
    facList = list(map(_del_fac_lable, facList))

    conDict = {}
    maxNum, maxName = 0, ''
    for f in facList:
        if f not in conDict.keys():
            conDict[f] = 1
        else:
            conDict[f] += 1
        if conDict[f] > maxNum:
            maxNum = conDict[f]
            maxName = f
    return maxName
    
def del_channel_fac(ch_id, pcnDB):
    get_value = lambda x: x["NAME"][0] if 'NAME' in x.keys() else None
    facList = get_value(pcnDB.get_fac_inf(ch_id))
    facOut = _del_fac_lable(facList)

    return facOut


def _del_fac_lable(facVal):
    if  not isinstance(facVal, str):
        return 'Others'
    key_words = ['华为', '依赛', '烽火', '马可尼', 'ECI北京', '中兴', '北电', '赛特']
    re_str = '|'.join(key_words)
    
    re_op_matchKeys = lambda x: [re.search(re_str, x), re.search('SDH', x)]
    re_op_delNone = lambda x: x.group(0) if x is not None else ''

    facValList = re_op_matchKeys(facVal)
    res_val = ''.join([re_op_delNone(i) for i in facValList])
    res_val = res_val if len(res_val) != 0 else facVal

    return res_val

def get_fac_inf_bu(pcnBussInf, pcnDB):
    buKey = list(pcnBussInf.keys())
    for b_idx in tqdm(range(len(buKey))):
        b_id = buKey[b_idx]
        busInfDict = pcnBussInf[b_id]
        fac = del_business_fac(busInfDict['CHANNELS'], pcnDB)
        busInfDict['FAC_LABLE'] = fac
    
    return pcnBussInf

def get_fac_inf_ch(channelInfPd, pcnDB):
    facSer = pd.Series(data=None, dtype='object')
    for i in tqdm(range(len(channelInfPd))):
        chIloc = channelInfPd.iloc[i]
        ch_id = chIloc['OBJ_ID']
        fac = del_channel_fac(ch_id, pcnDB)
        facSer = pd.concat([facSer, pd.Series([fac])], ignore_index=True)
    
    channelInfPd.loc[:,['FAC_LABLE']] = facSer

    return channelInfPd


    

In [8]:
pcnBussInf = get_fac_inf_bu(pcnBussInf, pcnTestDB)

100%|██████████| 18083/18083 [03:59<00:00, 75.61it/s]


In [9]:
channelInfPd =  get_fac_inf_ch(channelInfPd, pcnTestDB)

100%|██████████| 29259/29259 [06:35<00:00, 74.06it/s] 


## 3.2 获取通道类型和业务类型

In [10]:
def get_business_type(pcnBussInf, pcnDB):
    getInt = lambda x: x[0] if isinstance(x, list) else x
    for b_id, bussInfDict in pcnBussInf.items():
        b_type = pcnDB.get_buss_type(b_id)
        bussInfDict["B_TYPE"] = getInt(b_type['BUZ_TYPE']) if b_type['BUZ_TYPE'] != None else -1

    return pcnBussInf
    

In [11]:
def _del_cType(cList, channelInfPd):
    cTpyeList = []
    for cha in cList:
        res = channelInfPd.loc[channelInfPd['OBJ_ID'] == cha, "CHANNEL_TYPE"]
        cTpyeList.extend(res.to_list())
    return cTpyeList

def get_channel_type(pcnBussInf, channelInfPd):
    for b_id, bussInfDict in pcnBussInf.items():
        chaList = bussInfDict['CHANNELS']
        cTpyeList = _del_cType(chaList, channelInfPd)
        cType = cTpyeList[0] if len(cTpyeList) != 0 else '-1'
        bussInfDict['C_TYPE'] = cType

    return pcnBussInf
    

In [12]:
pcnBussInf = get_business_type(pcnBussInf, pcnTestDB)
pcnBussInf = get_channel_type(pcnBussInf, channelInfPd)

## 3.3 路由通道过滤

In [13]:
# 扫描站连接属性
def get_stnLink_attribute(channelInfPd, pcnBussInf):
    stnLinkInf = {}
    print("Collect channel type information")
    for i in tqdm(range(len(channelInfPd))):
        chInfIloc = channelInfPd.iloc[i]
        aStn, zStn = chInfIloc['A_STATION_NAME'], chInfIloc['Z_STATION_NAME']
        cType = chInfIloc['CHANNEL_TYPE']
        facType = chInfIloc['FAC_LABLE']
        stnKey = '{}-{}'.format(aStn, zStn)
        if stnKey not in stnLinkInf.keys():
            stnLinkInf[stnKey] = {'CTYPE': set([cType]), 'FTYPE': set([facType])}
        else:
            stnLinkInf[stnKey]['CTYPE'] = stnLinkInf[stnKey]['CTYPE'] | set([cType])
            stnLinkInf[stnKey]['FTYPE'] = stnLinkInf[stnKey]['FTYPE'] | set([facType])

    print("Collect business type information")
    buKey = list(pcnBussInf.keys())
    for b_idx in tqdm(range(len(buKey))):
        b_id = buKey[b_idx]
        bussInf = pcnBussInf[b_id]
        aStn, zStn = bussInf['A_STATION_NAME'], bussInf['Z_STATION_NAME']
        bType = bussInf['B_TYPE']
        # cType = chInfIloc['CHANNEL_TYPE']
        stnKey = '{}-{}'.format(aStn, zStn)
        if stnKey not in stnLinkInf.keys():
            stnLinkInf[stnKey] = {'BTYPE': set(bType)}
        else:
            if 'BTYPE' in stnLinkInf[stnKey].keys():
                stnLinkInf[stnKey]['BTYPE'] = stnLinkInf[stnKey]['BTYPE'] | set([bType])  
            else:
                stnLinkInf[stnKey]['BTYPE'] = set([bType])
        
    return stnLinkInf



In [14]:
stnLinkInf = get_stnLink_attribute(channelInfPd, pcnBussInf)

Collect channel type information


100%|██████████| 29259/29259 [00:01<00:00, 15136.83it/s]


Collect business type information


100%|██████████| 18083/18083 [00:00<00:00, 620307.34it/s]


In [15]:
def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))


def read_yaml_to_dict(yaml_path: str, ):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value

In [16]:
def filter_routes(routeDict, pcnBussInf, stnLinkInf):
    b_idList = list(routeDict.keys())
    TypeRule = read_yaml_to_dict('./conf/limit_rule.yml')
    routeLimitDict = {}
    for b_idx in tqdm(range(len(b_idList))):
        b_id = b_idList[b_idx]
        routes = routeDict[b_id]
        TypeDict = dict([(key, pcnBussInf[b_id][key]) for key in ['FAC_LABLE', 'B_TYPE', 'C_TYPE']])
        routesLimit = _del_routes_limit(routes, TypeDict, TypeRule, stnLinkInf)
        routeLimitDict[b_id] = routesLimit
        # break
    return routeLimitDict

def _del_routes_limit(routes, TypeDict, TypeRule, stnLinkInf):
    routesOut = []
    for routeList in routes:
        RoutePass = False
        if len(routeList) == 2:
            continue
        for a_idx in range(len(routeList) - 1):
            z_idx = a_idx + 1
            aStn, zStn = routeList[a_idx], routeList[z_idx]
            stnKey = '{}-{}'.format(aStn, zStn)
            stnKey1 = '{}-{}'.format(zStn, aStn)
            stnInfAZ = stnLinkInf[stnKey] if stnKey in stnLinkInf.keys() else stnLinkInf[stnKey1]
            stnInfAZ1 = stnLinkInf[stnKey1] if stnKey1 in stnLinkInf.keys() else stnLinkInf[stnKey]
            stnInfAZ.update(stnInfAZ1)

            #BTYPE
            if 'BTYPE' in stnInfAZ.keys():
                bTypeRule = TypeRule['BTYPE']
                stnBType = stnInfAZ['BTYPE']
                bType = TypeDict['B_TYPE']
                bRes = stnBType & set(bTypeRule[bType])
                if len(bRes) == 0:
                    # print(stnBType, bTypeRule[bType], bType)
                    # print('BTYPE out')
                    break

            #CTYPE
            cTypeRule = TypeRule['CTYPE']
            stnCType = stnInfAZ['CTYPE']
            cType = TypeDict['C_TYPE']
            # if cType == -1:
                # print(TypeDict)
                # print(stnInfAZ)
                # print(cTypeRule)

            cRes = stnCType & set(cTypeRule[cType])
            if len(cRes) == 0:
                break

            #FAC_LABLE
            if 'FTYPE' in stnInfAZ.keys():
                facType = TypeDict['FAC_LABLE']
                if facType == 'Others' or 'Others' in stnInfAZ['FTYPE']:
                    continue
                if facType not in stnInfAZ['FTYPE']:
                    # print(facType, stnInfAZ['FTYPE'])
                    # print('FTYPE out')

                    break
            if zStn == routeList[-1]:
                RoutePass = True

        if RoutePass == True:
            routesOut.append(routeList)
    return routesOut
        

def _del_single_route(azStn, stnDict, TypeDict):
    pass

routeLimitDict = filter_routes(routeDict, pcnBussInf, stnLinkInf)

100%|██████████| 18083/18083 [00:06<00:00, 2711.79it/s]


## 3.4 获取通道优选数据

In [17]:
# 获取路由跳数
def get_routes_steps(routeLimitDict):
    routeStepsDict = {}
    b_keyList = list(routeLimitDict.keys())
    for b_idx in tqdm(range(len(b_keyList))):
        b_id = b_keyList[b_idx]
        routes = routeLimitDict[b_id]
        routeSteps = []
        for route in routes:
            rSteps = len(route)
            routeSteps.append(rSteps)
        routeStepsDict[b_id] = np.array(routeSteps)

    return routeStepsDict
routeStepsDict = get_routes_steps(routeLimitDict)

100%|██████████| 18083/18083 [00:00<00:00, 281373.37it/s]


In [ ]:
pd_tmp = pd.DataFrame(data=stnDict)
pd_tmp = pd_tmp.T.reset_index(drop=True)
pd_tmp.loc[pd_tmp['VOLTAGE_CLASS'] == '11']


In [19]:
stnInfPd = pd.DataFrame(data=stnDict)
stnInfPd = stnInfPd.T.reset_index(drop=True)

def get_route_attr(route, stnInfPd, attr):
    attrRes = []
    for stn in route:
        resAttr = stnInfPd.loc[stnInfPd['STN_NAME'] == stn, attr].values
        if len(resAttr) == 0:
            resAttr_int = 5 if attr == 'VOLTAGE_CLASS' else 3
        elif resAttr[0] is None:
            resAttr_int = 4
        else:
            resAttr_int = float(resAttr[0])
            if attr == 'VOLTAGE_CLASS':
                resAttr_int = 0.5 if resAttr_int == 15 else resAttr_int
                resAttr_int = 5 if resAttr_int == 16 else resAttr_int
        attrRes.append(resAttr_int)

    return np.array(attrRes, dtype=np.float64)

In [26]:
# 获取路由调度等级
def get_routes_DSP(routeLimitDict, stnInfPd):
    routeDSPDict = {}
    b_keyList = list(routeLimitDict.keys())
    for b_idx in tqdm(range(len(b_keyList))):
        b_id = b_keyList[b_idx]
        routes = routeLimitDict[b_id]
        routesDSP = []
        for route in routes:
            rDSPs = get_route_attr(route, stnInfPd, 'VOLTAGE_CLASS')
            routesDSP.append(rDSPs)
        routeDSPDict[b_id] = routesDSP

    return routeDSPDict 

routeDSPDict = get_routes_DSP(routeLimitDict, stnInfPd)


100%|██████████| 18083/18083 [03:43<00:00, 81.03it/s] 


In [ ]:
routeDSPDict

In [22]:
# 获取站点电压等级
def get_route_Vol(routeLimitDict, stnInfPd):
    routeVolDict = {}
    b_keyList = list(routeLimitDict.keys())
    for b_idx in tqdm(range(len(b_keyList))):
        b_id = b_keyList[b_idx]
        routes = routeLimitDict[b_id]
        routesVol = []
        for route in routes:
            rVol = get_route_attr(route, stnInfPd, 'DSP_LEVEL')
            routesVol.append(rVol)
            
        routeVolDict[b_id] = routesVol

    return routeVolDict

routeVolDict = get_route_Vol(routeLimitDict, stnInfPd)


100%|██████████| 18083/18083 [04:29<00:00, 66.98it/s] 


In [ ]:
routeVolDict

# 4. 路由优选

## 4.1 路由优选数据计算

In [24]:
def route_optim(routeStepsDict, routeDSPDict, routeVolDict):
    ## 直接调用模糊层次分析模块
    routeScoreDict = {}
    b_key = routeStepsDict.keys()
    for b_id in b_key:
        DSPList, VolList = routeDSPDict[b_id], routeVolDict[b_id]
        DSPm = np.array([i.mean() for i in DSPList])
        Volm = np.array([i.mean() for i in VolList])
        Steps = routeStepsDict[b_id]
        
        Weight = read_yaml_to_dict('./conf/factor_weight.yml')
        scores = DSPm * Weight['DSP_LEVEL'] + Volm * Weight['VOLTAGE_CLASS'] + Steps * Weight['STEP']

        routeScoreDict[b_id] = scores
        

    return routeScoreDict

routeScoreDict = route_optim(routeStepsDict, routeDSPDict, routeVolDict)

In [25]:
# 输出最佳路由
def get_optimum_route(routeScoreDict, routeLimitDict):
    optRoutes = {}
    for b_id in routeScoreDict:
        routeLimit = routeLimitDict[b_id]
        routeScore = routeScoreDict[b_id]

        if len(routeLimit) > 0:
            minIdx = np.argmin(routeScore)
            optRoute = routeLimit[minIdx]
        else:
            optRoute = None

        optRoutes[b_id] = optRoute

    return optRoutes


optRoutes = get_optimum_route(routeScoreDict, routeLimitDict)

# -1.测试

In [83]:
empty_bid = []
num_bid = []
for b_id, routes in routeLimitDict.items():
    num_bid.append(len(routes))
    if routes.__len__() == 0:
        empty_bid.append(b_id)
empty_bid.__len__()

7878

In [81]:
e_len_num, e_same_type = 0, 0
e_len_d, e_same_d ={}, {}
for b_id, d in pcnBussInf.items():
    c_type = d['C_TYPE']
    cList = d['CHANNELS']
    if len(cList) != len(c_type):
        e_len_num += 1
        e_len_d[b_id] = d
    for i in range(len(c_type)):
        if i == 0:
            continue
        else:
            if c_type[i-1] != c_type[i-1]:
                e_same_type += 1
                e_same_d[b_id] = d
                


print("Scan pcnBussInf:\n\t ERRO_LEN:{}\n\tERRO_TYPE:{}".format(e_len_num, e_same_type))


Scan pcnBussInf:
	 ERRO_LEN:1368
	ERRO_TYPE:0


In [99]:
pcnBussInf['F07A0A01-9FA5-4D65-AE19-ADA4FF7219C6-33276']

{'A_STATION_NAME': '龙岗电厂',
 'Z_STATION_NAME': '500kV花都变',
 'CHANNELS': ['0CFBF67E-4699-4808-A4A6-81FC829CF4CD-00664',
  'CD996599-B045-43B0-9785-F2B51C807217-43869',
  '0E0BBF9D-2FA1-40AF-949D-7D046035E6DF-20996'],
 'FAC_LABLE': '华为SDH',
 'B_TYPE': '3',
 'C_TYPE': '1'}

In [18]:
pcnDBInf={
        'USER': 'PCN_TEST',
        'PASS': '784427618',
        'SID': '172.17.0.2:1521/LHR11G'
    } 
print(" PCN DataBase INF ".center(50, '*'))
for k in pcnDBInf.keys():
    if k != 'PASS':
        print("\t* {}: {}".format(k, pcnDBInf[k]))
print(" Successfully connected! ".center(50, '*'))


**************** PCN DataBase INF ****************
	* USER: PCN_TEST
	* SID: 172.17.0.2:1521/LHR11G
************ Successfully connected! *************


In [80]:
import yaml

def save_dict_to_yaml(dict_value: dict, save_path: str):
    """dict保存为yaml"""
    with open(save_path, 'w') as file:
        file.write(yaml.dump(dict_value, allow_unicode=True))


def read_yaml_to_dict(yaml_path: str, ):
    with open(yaml_path) as file:
        dict_value = yaml.load(file.read(), Loader=yaml.FullLoader)
        return dict_value

my_config_dict = {
        "mysql": {
            "host": "127.0.0.1",
            "tables": ["table_1", "table_2"],
        },
        "redis": {
            "host": "127.0.0.1",
            "db": 3,
        }
    }

TypeRule={
    'BTYPE':
    {
        '1':['1', '2', '10', '18'],
        '2':['1', '2', '10', '18'],
        '10': ['1', '2', '10', '18'],
        '18': ['1', '2', '10', '18'],
        '3':['3', '7', '8'],
        '7':['3', '7', '8'],
        '8':['3', '7', '8'],
        '4':['4', '6', '12', '16', '19'],
        '6':['4', '6', '12', '16', '19'],
        '12':['4', '6', '12', '16', '19'],
        '16':['4', '6', '12', '16', '19'],
        '19':['4', '6', '12', '16', '19'],
        '5':['5', '11'],
        '11':['5', '11'],
        '7':['7', '9'],
        '9':['7', '9'],
        '14':[str(i) for i in range(1,20)],
        '13':[str(i) for i in range(1,20)],
        '15':[str(i) for i in range(1,20)],
        '17':[str(i) for i in range(1,20)],
    },
    'CTYPE': dict((str(num), [str(num), '-1']) for num in range(1, 7))
}
TypeRule['CTYPE']['-1']= [str(i) for i in range(1,7)]
    
save_dict_to_yaml(TypeRule, "limit_rule.yml")
config_value = read_yaml_to_dict("limit_rule.yml")


In [28]:
Weight={
    'STEP': 0.5,
    'DSP_LEVEL': 0.2,
    'VOLTAGE_CLASS': 0.3
}
save_dict_to_yaml(Weight, "./conf/factor_weight.yml")
testW = read_yaml_to_dict('./conf/factor_weight.yml')

In [29]:
pcnDBInf={
    'USER': 'PCN_TEST',
    'PASS': '784427618',
    'SID': 'localhost:1521/LHR11G'
}
save_dict_to_yaml(pcnDBInf, "./conf/pcnDB_inf.yml")
testW = read_yaml_to_dict('./conf/pcnDB_inf.yml')
testW

{'PASS': '784427618', 'SID': 'localhost:1521/LHR11G', 'USER': 'PCN_TEST'}

In [39]:
test = np.array([2,1,3,4,5,6,6])
np.argmin(test)

1

In [ ]:
from utools.pickle_io import *
load_obj('routeScoreDict')